# *Uncertainty Propagation*: Polynomial approximation and interpolation (Tutorial 1)

### Problem statement
A key problem statement in the field of Uncertainty Quantification is _Uncertainty Propagation_. In general, such problems consist of the following ingredients:

* A vector of uncertain inputs, expressed as random-variables: $X = (X_1, X_2, \dots, X_d)$;
* A probability density function $p(X)$ describing the (possibly correlated) distribution of the inputs;
* A function $u(X)$ respresenting the action of (e.g.) our CFD code on the inputs.

Our goal is to estimate statistics (mean, variance, etc.) of the random-variable $Y = u(X)$.  We assume the following:

* $X_1, \dots, X_d$ are distributed *independently*, $X_i \perp X_j,\; i\neq j$, so $p(X) = p_1(X_1) p_2(X_2) \cdots p_d(X_d)$;
* We basically know everything about $p(X)$, e.g. its moments, its pdf, its shape, etc.

### Application examples
We will consider a simple test function for convenience. Let $X_1$ and $X_2$ be uniformly distributed on $[0, 1]$ and consider
$$
    u(X) = \left(\frac{1}{50} + \left(X_1 - \frac{7}{12}\right)^4\right)^{-1} \left(\frac{1}{25} + \left(X_2 - \frac{2}{3}\right)^2\right)^{-1}
$$
Our goal is to determine the mean of $u(X)$. As $u$ is known explicitly in this case (which is for CFD not the case), we know that:
$$
    \mathbb{E}[u] := \int_0^1 \int_0^1 u(x_1,x_2) p(x_1,x_2)\,\mathrm{d}x_1\,\mathrm{d}x_2 = 421.864040287311503221212881103111494455415403225767785...
$$

Contours of the function $u(X)$ are as follows (we also include helper libraries here, so always start with executing the following cell):

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.interpolate as interpolate

def u(x1, x2): return 1/(1/50 + (x1 - 7/12)**4) * 1/(1/25 + (x2-2/3)**2)

In [ ]:
x = np.arange(0, 1, 0.025)
y = np.arange(0, 1, 0.025)
X, Y = np.meshgrid(x, y)
Z = u(X, Y)
plt.figure()
CS = plt.contour(X, Y, Z)

## Monte Carlo

The Monte Carlo approach consists of sampling from $X_1$ and $X_2$ and evaluating $u(X)$ there. So a crude approximation of $\mathbb{E}[u]$ is the following:

In [ ]:
X1 = np.random.rand(10, 1)
X2 = np.random.rand(10, 1)

np.mean(u(X1, X2))

**EXERCISE 1**

Say $E^\text{MC}_N$ is the mean determined in this way using $N$ samples. Extend the snippet above to determine a list with means determined using $N = 10^0, \dots, 10^7$ samples. For each estimate, determine

$$e_N = \left|E^\text{MC}_N - \mathbb{E}[u]\right|$$

Create a $N - e_N$ log-log plot. What do you observe? Do you expect $e_N \to 0$ for $N \to \infty$? What is the *rate* of convergence?  Note: The `scipy.stats` function `linregress()` is a handy way of fitting a line to data - just remember to pass it the log-log values!  The first returned value is the slope of the fit.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

You may see that the Monte-Carlo estimate is noisy - this is of course inherent to the method!  If you want to see the standard-deviation at each $N$, you could (extra credit): run MC multiple times at each value of $N$; compute the sample variance, and then standard-deviation.  You can then plot $\mu \pm \sigma$ on your convergence plot.

## Interpolation: "Probabilistic collocation"

Monte Carlo almost always works, irrespective of the regularity (i.e. smoothness) of $u(\cdot)$, with a convergence rate *independent* of the dimension of the input.  Both these properties are extremely unusual for a numerical method.  More typical is the following: a method with a convergence rate of e.g. $\epsilon \propto 1/N^p$ in 1d ($p$th-order), has a rate of $\epsilon \propto 1/\sqrt[d]{N^p}$ in $d$-dimensions (assuming a tensor-product generalization - Q: Consider why).  So a 1st-order method ($p=1$ will be equal or worse than Monte-Carlo in $\geq 2$-dimensions.  A 2nd-order method ($p=2$) will be equal or worse in $\geq 4$-dimensions.

However, especially if accurary requirements are strict, Monte-Carlo requires an excessive number of function evaluations.  This is especially onerous if a single function evaluation takes a very long time (the case of CFD!).  A different approach, as discussed in the lectures, is stochastic polynomial quadature.

**EXERCISE 2.0**

We start with a simple, 1D example. Let $X_2 = 2/3$ and let (again) $X_1$ be uniformly distributed between $[0, 1]$. Then:
$$
\mathbb{E}[u] := \int_0^1 u(x, 2/3) \,\mathrm{d} x = 913.2377633133653539471472717582753...
$$
Since the background (input) distribution is uniform, we know the Gauss-Legendre quadrature rule is optimal, and the Legendre polynomials form an orthogonal basis.  As in the lectures `numpy` implements this rule, but on the interval $[-1,1]$.  Scale the nodes and weights to the required interval $[0,1]$.


In [ ]:
def scaled_gausslegendre(M):
    """Return the Gauss-Legendre rule with M+1 nodes on [0,1]"""
    x, w = np.polynomial.legendre.leggauss(M+1)
    raise NotImplementedError()  ### TODO Implement scaling of weights and nodes
    
    return x, w

def quad_gausslegendre(f, M):
    """Return the Gauss-Legendre approximation of the integral of f on [0,1]"""
    raise NotImplementedError()  ### TODO Implement quadrature rule

You can now plot the test function, sampled at your nodes, and interpolated with a polynomial there.  We use the `scipy` function `BarycentricInterpolator(x, y)`, which returns a function (a polynomial) which interpolates the input data.  (If you're not sure how it does this, implement a version yourself!)

https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.BarycentricInterpolator.html

In [ ]:
x, w = scaled_gausslegendre(5)
def u_1d(x): return u(x, 2/3)
Y = u_1d(x)
uhat = interpolate.BarycentricInterpolator(x, Y)

In [ ]:
plt.figure()
xx = np.arange(0, 1, 0.01)

plt.plot(xx, u_1d(xx))
plt.plot(xx, uhat(xx))
plt.scatter(X, Y, c='black')

Let $\hat{u}_N$ be the polynomial in the figure above. The following integral can now be determined exactly (!):
$$
    \mathbb{E}[\hat{u}_N] = \int_0^1 \hat{u}_N(x) \,\mathrm{d} x \simeq \mathbb{E}[u]
$$

You may use the following function for this (here $M = 15$):

In [ ]:
print( quad_gausslegendre(u_1d, 15) )

**EXERCISE 2.1**

Say $E^\text{IP}_M$ is the mean determined in this way using $M+1$ evaluations of the function using the Gauss-Legendre quadrature rule.  Plot again convergence of this rule for this problem.  That is, for a sequence of $M$s determine:

$$e_M = \left|E^\text{IP}_M - \mathbb{E}[u]\right|.$$

Create a $N$ against $e_N$ log plot. What do you observe? Does $e_N \to 0$ hold (for $N \to \infty$)? What is the rate of convergence? How many nodes do you need to reach an error of $10^{-8}$?  And an error of $1$%?

The original function has **two** variables. But that is not that complex, we can simply use the tensor-product idea, and apply our interpolation procedure twice in a loop as follows:

In [ ]:
M1, M2 = 5, 6
x1, w1 = scaled_gausslegendre(M1)
x2, w2 = scaled_gausslegendre(M2)

sum = 0
for i in range(M1+1):
    for j in range(M2+1):
        sum += u(x1[i], x2[j])*w1[i]*w2[j]

sum

**EXERCISE 3**

Create a similar plot as in Exercise 1 and 2: plot $N - e_N$, where $N$ is the *total* number of nodes used and $e_N$ is the error. What do you oberserve? $e_N \to 0$? What is (approximately) the rate of convergence?  How many nodes do you need to reach an accuracy of 1%?  **Note:** it's useful here to make a function called `quad2d(f, M1, M2)` that encapsulates the code in the cell above (performs the numerical quadrature in 2d and returns the answer), so you can call it for a variety of $M_1$ and $M_2$.

## Smolyak Sparse grid - Advanced material

In multiple dimensions, a vastly improved way of performing the interpolation procedure is the Smolyak sparse grid concept.  In this tutorial, you will implement your own sparse grid, based on your tensor-product formula from the previous tutorial.  The same function as above is used initially, but then we move on to XFoil and attempt to evaluate statistics of surface pressure due to uncertain onflow conditions.  Remember to execute the notebook up to this point, to initialize `u()`, your `quad2d()`, etc.

### Clenshaw-Curtis quadrature
Remember for sparse grids to work well, we need to base them on 1d rules with *nested* nodes.  The 1-dimensional rule constructed above (Legendre) is not nested.  The Clenshaw-Curtis rule *is* nested - the nodes and weights are implemented in the function below.  Verify (by plotting) the nesting property for these rules, namely that if $\mathcal{X}_M = \{x_0,\dots,x_M \}$ is the set of nodes of the $M$-th rule, then
$$
\mathcal{X}_0 \subset \mathcal{X}_2 \subset \mathcal{X}_4 \subset \mathcal{X}_8 \subset \cdots \subset \mathcal{X}_{2^j}.
$$
Perform also some basic checks on the rule, integrals of $1$, $x$, etc. correct.

**Note:** The function below computes the weights by integrating the Lagrange polynomials corresponding to the nodes.  Why does this work?

In [ ]:
def scaled_clenshaw(M):
    """Return nodes,weights of Clenshaw-Curtis quadrature with M+1 nodes on [0,1]"""
    if M == 0: return np.array([0.5]), np.array([1.]), # Special case
    x = np.cos(np.linspace(0, np.pi, M+1)[::-1])/2 + 1/2   # Nodes have an easy formula
    w = np.zeros(M+1)                                # Weights don't! So compute numerically!
    for i in range(M+1):
        lagrange = interpolate.BarycentricInterpolator(x, np.eye(M+1)[i])  # i-th Lagrange poly
        w[i] =  quad_gausslegendre(lagrange, M)
    return x, w

### Sparse grid construction
The Smolyak sparse grid formula we use is the following. Let  $\mathcal{Q}_M$ be the *one-dimensional* (or "univariate") quadrature rule developed in the previous section.  So
$$
\mathcal{Q}_M[f] := \sum_{i=0}^M w_i f(x_i) \simeq \int_0^1 f(x) \rho(x) \;\mathrm{d}x
$$
where $\rho(x)$ is the uniform density.  Let $\Delta_k$ be differences of two of these rules as follows:
$$
\Delta_0 := \mathcal{Q}_0, \quad\Delta_1 := \mathcal{Q}_1-\mathcal{Q}_1, \quad\Delta_2 := \mathcal{Q}_2 - \mathcal{Q}_1, \quad\Delta_3 := \mathcal{Q}_3 - \mathcal{Q}_2,\quad\dots
$$
so we can also evaluate e.g. $\Delta_M[f]$ and get some number.
Then the Smolyak grid we seek is:
$$S_K = \sum_{i + j \leq K} \left(\Delta_{g(i)} \otimes \Delta_{g(j)}\right),$$
with 
$$
g(j) = \begin{cases}0 &\text{if $j = 0$} \\ 2^j &\text{if $j\geq 1$}.\end{cases}
$$

Note that this $g(\cdot)$ is necessary because we don't want to use *every* degree $M$ of the quadrature rule.  One reason is that Clenshaw-Curtis is not perfectly nested in the sense that $\mathcal{X}_M \subset \mathcal{X}_{M+1}$ for all $M$.  The nested property only holds for the $M = 2^i$ rules.  So the $g(\cdot)$ function selects those rules only.  It is the mapping from the index of the sparse grid *level* to the degree of the rule $M$.

**Hint**: "$\otimes$" is linear, so
$$\Delta_m \otimes \Delta_n = (\mathcal{Q}_m - \mathcal{Q}_{m-1}) \otimes (\mathcal{Q}_n - \mathcal{Q}_{n-1}) = \mathcal{Q}_m \otimes \mathcal{Q}_n - \mathcal{Q}_{m-1} \otimes \mathcal{Q}_n - \mathcal{Q}_{m} \otimes \mathcal{Q}_{n-1} + \mathcal{Q}_{m-1} \otimes \mathcal{Q}_{n-1}$$

**EXERCISE 4**

Complete the following snippet for the function Smolyak, which takes a function and a $K$ and integrates that function using a sparse grid.  Note that you can compute the primitives $\mathcal{Q}_i\otimes\mathcal{Q}_j$ using your function `quad2d()` from the previous section (remember to change the quadrature rule to Clenshaw!).  Try implementing an equivalent function `quad2d_delta()` that computes $\Delta_m \otimes \Delta_n$ using these primitives, and use that function to construct the main sum above.

In [ ]:
def g(i):
    raise NotImplementedError()  # TODO Implement g(), defined above
    
def quad2d_delta(f, l1, l2):
    raise NotImplementedError()  # TODO Implement \Delta_l1 \otimes \Delta_l2, where l1, l2 are levels

def quad_smolyak(f, K):
    raise NotImplementedError()  # TODO Implement smolyak quadrature

If you did everything correct, the following should print $1$ (or $0.9999999$, or $1.00000001$) to the screen:

In [ ]:
quad_smolyak(lambda x, y: 1, 5)

**EXERCISE 5**

Extend the code of your smolyak grid such that it also returns the *nodes* where the function is evaluated.  For $K = 3, 4, \text{ and } 5$ scatter-plot your Smolyak grid.  It should look sparser than a tensor-product grid, but denser than a "one-at-a-time" grid.

**EXERCISE 6**

Redo EXERCISE 3 for the Smolyak sparse grid. Plot $N - e_N$ where $N$ is the number of *nodes* for $K = 1, \dots, 8$.  Note that computing the number of nodes is non-trivial, unlike for simple tensor-product rules.  It depends on the number of "collisions" of nodes of the component rules.

To compute the number of unique nodes, do the following: generate a `N x 2` numpy `array` of nodal coordinates `xx = array([ [0,1], [.5,.5], [.5,.5], ...])`, possibly with duplicates.  You should have this from EXERCISE 5 already.  Then apply `np.unique(xx, axis=0)` which returns a list of unique nodes, and count the length of the result.

Does $e_N \to 0$ hold again? Is this faster or slower than the tensor grid approach, or about the same?

## XFOIL Application

Such a test function is a good approach to assess whether your UQ methodology is performing according to theoretical expectations.  But the real deal is applying everything to a real function. In this section, we will apply the implemented methods to `XFOIL` and assess the effect of uncertainty in the angle of attack, Mach number, and Reyonds number on the pressure coefficient on the airfoil.

Let's start by including the provided `xfoil` module and obtain a cp-x figure:

In [ ]:
import xfoil as xfoil
xfoil.init()

plt.plot(xfoil.x, -xfoil.cp(5.1232, 0.35, 5e6))

As you can see, the pressure coefficient can be obtained by the function `cp` and the locations of each obtained `cp` is stored in `xfoil.x`. `cp` accepts three arguments in the following order: angle of attack $\alpha$, mach, and Reynolds number. The angle of attack $\alpha$ must be between 1 and 6. The Mach number must be between 0.3 and 0.6. The Reynolds number must be between $10^6$ and $10^7$. Any combination in these intervals should work. Any combination outside these intervals will most likely result into an error.

Summarized:
* $\alpha \in [1^\circ, 6^\circ]$
* $M_\infty \in [0.3, 0.6]$
* $Re \in [10^6, 10^7]$

**EXERCISE 8**

Using $N = 100$ random uniformly distributed samples in the $\alpha$ and $M_\infty$ intervals described above, and fix $Re = 10^6$.  Plot the mean and (mean $\pm$ standard deviation) of the pressure coefficient. Be aware of the fact that you *cannot* pass vector inputs to the function `cp()`, it accepts only 3 scalars.

**Hint:**
Use `np.random.uniform(a, b, 100)` to get 100 uniform random samples on the interval $[a, b]$.

**EXERCISE 9**

Use a 2d tensor grid to determine the mean and standard deviation of the `xfoil`-problem. It is not possible to plot the exact error as we did previously, since the exact value is not known. Therefore, create a plot that plots the consecutive difference:

$$e_n = \left\|\text{(Estimate with $(M+1)^2$ nodes)} - \text{(Estimate with $M^2$ nodes)}\right\|$$

where $\|\cdot\|$ is e.g. the $\ell^2$-norm.  Note that out quadrature is built for $\mathcal{U}(0,1)$, so some scaling will be necessary to apply it to $\mathcal{U}(1,6)$, etc.  You can scale the nodes/weights, or just scale the input to XFoil.

**EXERCISE 10**

Again fix $Re = 10^6$. Use your Smolyak sparse grid code to determine the mean and standard deviation of the `xfoil`-problem with this fixed Reynolds number.  Recreate the plot of the consecutive difference, and compare convergence with the tensor-product method.